## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pangnirtung,CA,66.1451,-65.7125,20.39,92,84,1.63,broken clouds
1,1,Port Blair,IN,11.6667,92.7500,81.48,75,19,6.17,few clouds
2,2,Sao Filipe,CV,14.8961,-24.4956,72.28,75,6,12.48,clear sky
3,3,Karasjok,NO,69.4719,25.5112,15.46,90,99,5.28,overcast clouds
4,4,Tuktoyaktuk,CA,69.4541,-133.0374,-22.49,100,99,7.05,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Port Blair,IN,11.6667,92.7500,81.48,75,19,6.17,few clouds
5,5,Vieux-Habitants,GP,16.0589,-61.7659,80.17,68,78,12.68,broken clouds
19,19,Bilma,NE,18.6853,12.9164,87.89,3,24,6.02,few clouds
29,29,Rikitea,PF,-23.1203,-134.9692,77.38,72,52,17.49,light rain
32,32,Kloulklubed,PW,7.0419,134.2556,79.45,80,100,3.42,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
684,684,Windhoek,NaN,-22.5594,17.0832,75.40,55,20,9.22,few clouds
689,689,Pangai,TO,-19.8000,-174.3500,81.21,79,100,12.84,overcast clouds
691,691,Arlit,NE,18.7369,7.3853,88.86,3,4,6.08,clear sky
695,695,Nueva Loja,EC,0.0847,-76.8828,75.22,94,75,8.05,broken clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                198
City                   198
Country                195
Lat                    198
Lng                    198
Max Temp               198
Humidity               198
Cloudiness             198
Wind Speed             198
Current Description    198
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [9]:
preferred_cities_df.count()

City_ID                195
City                   195
Country                195
Lat                    195
Lng                    195
Max Temp               195
Humidity               195
Cloudiness             195
Wind Speed             195
Current Description    195
dtype: int64

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Port Blair,IN,81.48,few clouds,11.6667,92.7500,
5,Vieux-Habitants,GP,80.17,broken clouds,16.0589,-61.7659,
19,Bilma,NE,87.89,few clouds,18.6853,12.9164,
29,Rikitea,PF,77.38,light rain,-23.1203,-134.9692,
32,Kloulklubed,PW,79.45,overcast clouds,7.0419,134.2556,
34,Bandarbeyla,SO,81.12,clear sky,9.4942,50.8122,
38,Richards Bay,ZA,75.42,clear sky,-28.7830,32.0377,
39,Bustamante,MX,80.62,few clouds,26.5333,-100.5000,
40,Atuona,PF,78.22,scattered clouds,-9.8000,-139.0333,
41,Victoria,HK,75.18,scattered clouds,22.2855,114.1577,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Port Blair,IN,81.48,few clouds,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
5,Vieux-Habitants,GP,80.17,broken clouds,16.0589,-61.7659,Gîtes Dardanelle
19,Bilma,NE,87.89,few clouds,18.6853,12.9164,Expeditions Ténére voyages
29,Rikitea,PF,77.38,light rain,-23.1203,-134.9692,People ThankYou
32,Kloulklubed,PW,79.45,overcast clouds,7.0419,134.2556,Storyboard Beach Resort
...,...,...,...,...,...,...,...
682,Mayumba,GA,78.98,light rain,-3.4320,10.6554,LIKWALE LODGE
689,Pangai,TO,81.21,overcast clouds,-19.8000,-174.3500,Ha'apai Beach Resort
691,Arlit,NE,88.86,clear sky,18.7369,7.3853,Case de Passage De la Prefecture
695,Nueva Loja,EC,75.22,broken clouds,0.0847,-76.8828,Limoncocha Great Hotel


In [16]:
hotel_df.count()

City                   195
Country                195
Max Temp               195
Current Description    195
Lat                    195
Lng                    195
Hotel Name             195
dtype: int64

In [19]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [20]:
clean_hotel_df.count()

City                   195
Country                195
Max Temp               195
Current Description    195
Lat                    195
Lng                    195
Hotel Name             195
dtype: int64

In [24]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))